### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Setting up authorization for to use forecasting APIs

In [1]:
from oci.signer import Signer

auth = Signer(
    tenancy='ocid1.tenancy.oc1..aaaaaaaaiyavtwbz4kyu7g7b6wglllccbflmjx2lzk5nwpbme44mv54xu7dq',
    user='ocid1.user.oc1..aaaaaaaacumalt7qfzkkf4iu2vgtgmery46ptv3jwybmr2ea4oerhzm6gmqa',
    fingerprint='de:58:ab:fa:13:af:26:06:f6:e4:3f:e8:04:47:09:9f',
    private_key_file_location='/home/datascience/.oci/api_key.pem',
)

# Install Necessary packages

In [4]:
!pip install pandas

  Using cached pandas-1.1.5-cp36-cp36m-manylinux1_x86_64.whl (9.5 MB)


In [6]:
!pip install matplotlib

  Using cached matplotlib-3.3.4-cp36-cp36m-manylinux1_x86_64.whl (11.5 MB)
  Using cached kiwisolver-1.3.1-cp36-cp36m-manylinux1_x86_64.whl (1.1 MB)


In [8]:
!pip install simplejson

     |████████████████████████████████| 129 kB 21.9 MB/s            


# Data Preparation 

In [9]:
import pandas as pd
import requests
import json
import ast
import matplotlib.pyplot as plt
import re
import os
import simplejson

In [14]:
# Load data in notebook

df_primary = pd.read_csv('/home/datascience/ai-forecasting-demo/data/favorita_13_beverages_primary_v1.csv')
df_add = pd.read_csv('~/ai-forecasting-demo/data/favorita_13_beverages_add_v1.csv')

In [15]:
df_primary.head()

,item_id,date,sales
0,13_BEVERAGES,2013-01-01 00:00:00,0
1,13_BEVERAGES,2013-01-02 00:00:00,767
2,13_BEVERAGES,2013-01-03 00:00:00,987
3,13_BEVERAGES,2013-01-04 00:00:00,652
4,13_BEVERAGES,2013-01-05 00:00:00,1095


In [16]:
df_add.head()

,item_id,date,onpromotion
0,13_BEVERAGES,2013-01-01 00:00:00,0
1,13_BEVERAGES,2013-01-02 00:00:00,0
2,13_BEVERAGES,2013-01-03 00:00:00,0
3,13_BEVERAGES,2013-01-04 00:00:00,0
4,13_BEVERAGES,2013-01-05 00:00:00,0


### Convert the date field to "yyyy-mm-dd hh:mm:ss" format 

In [24]:
# modify date format
df_primary['date'] = pd.to_datetime(df_primary['date'],format='%Y-%m-%dT%H:%M:%S').apply(lambda x: str(x))

# modify date format
df_add['date'] = pd.to_datetime(df_add['date'],format='%Y-%m-%dT%H:%M:%S').apply(lambda x: str(x))

In [26]:
# Sort data
df_primary.sort_values(by = "date" , inplace = True)  
df_add.sort_values(by = "date" , inplace = True)

In [43]:
#primary data
prim_load = df_primary.values.transpose().tolist()
prim_load

[['13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVE

In [44]:
#additional data
add_load = df_add.values.transpose().tolist()
add_load

[['13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVERAGES',
  '13_BEVE

# Create AI Forecasting Project ID

Once the data is prepared , we will create the forecasting service project.

In [ ]:
url = "https://forecasting.---------------------.oraclecloud.com/20220101/projects"

payload = json.dumps({
  "displayName": "Forecast API Demo",
  "compartmentId": "ocid-------------------",
  "description": "Forecasting service API Demo",
  "freeformTags": None,
  "definedTags": None,
  "systemTags": None
})
headers = {
  'Content-Type': 'application/json'
}
response = requests.request("POST", url, headers=headers, data=payload, auth=auth)